- 保监会 八分类模型 1 预处理

# 基本设置

In [1]:
import jieba
import sys
import re
import time
import string

%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor
import os
import json
import urllib

import warnings
warnings.filterwarnings('ignore')

# 导入数据

In [2]:
folder = 'raw_data' # 数据文件夹
file_list = ['20180808_circ_class_revise.xlsx',]

## title + url

In [3]:
cor_data_raw = pd.DataFrame()
for file_name in file_list:
    tmp_data = pd.read_excel('data/{0}/{1}'.format(folder, file_name))
    print(tmp_data.shape, file_name)
    print()
    
    cor_data_raw = pd.concat([cor_data_raw, tmp_data], axis = 0)
    
print('总数：', cor_data_raw.shape)
cor_data_raw.head()

(953, 4) 20180808_circ_class_revise.xlsx

总数： (953, 4)


,title,url,publishtime,name
0,我国政策性信用保险承保金额突破5000亿美元-财急送网,http://www.caijisong.com.cn/article/news-27344...,2018-01-04 00:00:00,行业
1,【保险业开门红战报：总保费下滑，价值保费获增长】(分享自万得股票)@Wind资讯 ​\n\t\t,http://weibo.com/1348660061/FDsNUujBP,2018-01-08 23:09:01,行业
2,因理论不足及司法配合不利 车险反欺诈任重道远,http://www.gzjrw.com.cn/Item/335756.aspx,2018-01-04 00:00:00,行业
3,30余家互联网保险融资逾10亿 涉互联网保险但无牌照,http://www.gzjrw.com.cn/Item/335761.aspx,2018-01-04 00:00:00,行业
4,短期趸交保险产品销售紧俏 一小时揽收超10亿元保费,http://www.gzjrw.com.cn/Item/335762.aspx,2018-01-04 00:00:00,产品销售


## content

In [4]:
if 1:    
    raw_data = []
    for index in cor_data_raw.index:
        print(index)
#         if index % 1000 == 0:print('index: ', index)
        try :
            label = cor_data_raw.loc[index, 'name']
            title = cor_data_raw.loc[index, 'title']
            row = cor_data_raw.loc[index, 'url']
            url = "http://121.42.224.166/extract?url=" + row +"?j=1"
            #print(url)
            request = urllib.request.Request(url)
            response = urllib.request.urlopen(request)
            pageCode = response.read().decode('utf-8')
            content = pageCode[pageCode.index("Content") + 8:]
            title_url = pageCode[pageCode.index("Title")+7:pageCode.index("Source")-3]

            raw_data.append([label, title, content, row, title_url])
        except Exception as e:
            print('---------------')
            print(index, label, title)
            print(row)
            print(e)
            continue

    raw_data = pd.DataFrame(raw_data, columns = ['label', 'title', 'content', 'url', 'title_url'])
    filename = 'circ_class_combined_201800808_revise.xlsx'
    raw_data.to_excel('data/raw_data/%s'%filename, index = False)
    print(raw_data.shape)
    raw_data.head()

0
1
---------------
1 行业 【保险业开门红战报：总保费下滑，价值保费获增长】(分享自万得股票)@Wind资讯 ​
		
http://weibo.com/1348660061/FDsNUujBP
substring not found
2
3
4
5
6
7
8
9
10
11
12
---------------
12 资本市场 
		#港股y模型分析# $中国平安 hk02318$ 上周五点评2318本周仍有压力。窝轮资金仍集中于平保，认购证流入51.08（百万元）。今日点评：短期支持位在81元。港股解码Victor分析，目前在78-83之间上落。目前股价跌1.39%，股价81元。
			...展开全文c
		
http://weibo.com/3876076652/FDpxaglg7
substring not found
13
14
15
16
17
18
19
20
---------------
20 公司内部管理 降低社会保险缴费率，减轻小微企业负担
http://www.hndca.org.cn/news.asp?id=9965
substring not found
21
---------------
21 资本市场 
		【沪股通2018年首度净卖出贵州茅台 万科A等地产股也被净卖出】1月8日，贵州茅台股价再创新高，沪股通趁机净卖出2.02亿元，系2018年以来首次净卖出。地产股中，保利地产、万科A分别遭北上资金净卖出4693万元、1179万元，另外中国平安、平安银行等均被净卖出。 ​
		
http://weibo.com/2152990831/FDsm3DGma
substring not found
22
---------------
22 资本市场 
		今天地产股仍然表现强势，沪指也得以收上3409点，但有一点很清楚，目前股市资金还是存量的资金，当要炒作一个新的板块时，只能抛其它股票取得资金，今天中国平安和京东方为首的各自板块被抛售就是因为这。今天地产股大蓝筹万科和金地集团都收出一个长上影阳线，表明已经有逢高卖盘了，结合它们离5日线 ​
			...展开全文c
		
http://weibo.com/5458827367/FDqMG8Vy5
substring not found
23
24
---------

180
---------------
180 行业 中国银行淄博高新支行成功叙做首笔政银保业务
http://www.luxunw.com/info/ShowNews_35291.html
substring not found
181
182
183
---------------
183 公司内部管理 打破4大迷思 房贷寿险胜车险
http://www.msn.com/zh-tw/money/topstories/打破4大迷思-房貸壽險勝車險/ar-BBHPZDr?li=BBqiQ9P
'ascii' codec can't encode characters in position 59-60: ordinal not in range(128)
184
---------------
184 产品销售 保险是一个重要而不紧急的商品，所以有的人们在规划的时候不积极，这其中就有一部分人熬过风险，在风险来临时都还没有来得及准备充足的保险以转移风险。保险不能阻止风险的发生，但是可以把风险所造成的损失转移给保险公司。切记，条件允许的时候，保险规划要趁早！ ​
http://weibo.com/1258971190/FCMdM8xcU
substring not found
185
---------------
185 公司内部管理 【沈阳大额医保补助保险费缴费标准每人每年132元】1月3日，沈阳市人社局下发了关于缴纳2018年灵活就业医疗保险大额医疗费用补助保险费的通知，灵活就业参保人员大额医疗保险费用补助保险费实行按年缴纳，缴费标准为每人每年132元。另外，沈阳市人社局、市财政局下发了“关于调整我市城镇居民大病保险相 ​​​
http://weibo.com/1084443194/FCMdNAGeU
substring not found
186
---------------
186 公司内部管理 『保监会：保险资金设立股权投资计划不得直接或变相开展通道业务，不得开展嵌套投资 - 21财经』http://t.cn/RHmLKdg ​
http://weibo.com/3063367827/FCWsi98oU
substring not found
187
188
---------------
188 产品销售 
		亲爱的新老朋友们，你们的住院医疗保险要升级

263
---------------
263 公司内部管理 刚开年，帮自己买了重疾险和意外险。投资策略进行了调整，原先买的基金投资份额降低，分别再投资了其它3只基金，并且把原来那只基金投资进去的钱拿出来一部分买了一只股票基金。今年比较傻的事情是在什么都不知道的情况下申购了可转债，我真的是个呆子啊。 今年再帮自己买一个医疗险，然后养好身体，想办法多赚钱，变得美美的。今年牙套应该可以下了，然后考虑下巴的事情。不管别人怎么说，怎么怎么不可能，我始终坚持自己的道路走，让说不可能的人去不可能吧。
http://weibo.com/1775981135/FCP1ReQEG
substring not found
264
265
---------------
265 公司内部管理 青啤内忧外患财务投资人总难得志  复星66亿接棒迎来“美好生活”？
http://www.zmoney.com.cn/shangshi/show.php?itemid=1488
substring not found
266
---------------
266 行业 在前段时间密集调研过大消费后，最近三个月，机构不约而同地对整体估值水平较低的文化传媒板块兴趣大增，影视、游戏类上市公司成为重点关注对象。过去一个月时间内，传媒板块内有20余家上市公司接受了机构调研，涉及超过120家券商、基金、险资等机构。接受调研的公司中，游戏公司占比最高。（上海证券报）
http://weibo.com/3986703386/FCLflenlx
substring not found
267
268
269
270
---------------
270 公司内部管理 【保监会整治险资股权投资风险 禁止“名股实债”】在近年债券违约事件频发、“名股实债”风险聚集的情况下，1月5日，保监会对外发布《关于保险资金设立股权投资计划有关事项的通知》，对保险资产管理机构设立股权投资计划业务进行规范，防范保险资金以通道、嵌套、名股实债等方式开展股权投资计划业务 ​
http://weibo.com/1620607681/FDrfn2s41
substring not found
271
272
273
274
---------------
274 资本市场 《中国平安第二次放量暴跌》
12月27，中国平安，这个大白马，被市场喧

440
441
---------------
441 公司内部管理 招商安博保本混合型证券投资基金招募说明书（更新）摘要
http://zqrb.ccstock.cn/html/2018-01/06/content_139242.htm
substring not found
442
443
---------------
443 行业 上半年四大上市险企 保费同比均增22.6%
http://mheyongjiang.cn/kejixinwen/2018/01/06/795662.html
substring not found
444
---------------
444 资本市场 保监会进一步加强 险企股权信息披露
http://mheyongjiang.com.cn/shenghuobaike/2018/01/06/795793.html
substring not found
445
---------------
445 行业 互联网保险监管出台 第三方网站不得代收保费-宣城新闻网
http://lupan1.com.cn/szyw/2018/01/06/802304.html
substring not found
446
447
448
449
450
451
452
453
454
---------------
454 公司内部管理 险资股权投资计划禁设明确预期回报
http://epaper.financialnews.com.cn/jrsb/html/2018-01/06/content_244416.htm
substring not found
455
456
457
458
---------------
458 行业 商业车险改革试点落地　超七成消费者保费下降-五大连池新闻网
http://hliuzhicong.com.cn/house/2018/01/06/801918.html
substring not found
459
460
461
462
463
---------------
463 行业 内地访客新增保费大降六成  去香港买保险适合你最重要
http://finance.imgmg.com/article/73755.html
substring not found
464
465
466
----------

641
---------------
641 行业 
		风险正是因为其发生概率、发生时间的不确定性，所以往往会让人忽视风险的缴付在。很多朋友在缴了几年的保费后，发现好像并没有发生什么风险事件，此时的风险意识下降，觉得保险没什么用，又会重新思考是否有必要买保险。但风险往往就是在你不经意间就来了！ ​
		
http://weibo.com/1791815645/FDnzR6NqP
substring not found
642
643
---------------
643 行业 保险业严监管仍在持续 新年首周开出千万罚单　
http://www.morningpost.com.cn/2018/0108/1682878.shtml
substring not found
644
---------------
644 行业 
		看了几份报告，一季度保险行业可能保费会同比下滑，这可能是最近保险股连续下跌的主因。长期来看保险行业是非常不错的，不过去年涨幅巨大，今年一季度报如果不理想，今年可能还是要调调。 ​
		
http://weibo.com/1217536527/FDnKfhQwO
substring not found
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
---------------
660 行业 
		据说保险一季度开门黑，保费增长低于预期 ​
		
http://weibo.com/3972729684/FDo3ti0qh
substring not found
661
662
663
664
---------------
664 行业 
		#汽车洋葱圈# 不发生出险事故就没事儿了？这些违章行为照样会让你的保费上涨！ |这些违章除... ​
		
http://weibo.com/5310047613/FDoebyGxd
substring not found
665
666
667
668
---------------
668 行业 
		车险公司应该开发一个智能小设备，安装在车子某角落。可以记录车子使用频率、超速行为、急加速、急减速、急变道等数据，用这些数据结合该车报保险的数据、车辆被交警队扣分的数据，可以搭建一个针对单一车辆单独报价的保险公式，能把遵

751
752
---------------
752 公司内部管理 粤高速A：有战略投资保险公司的考虑还在等待合适的切入点
http://www.erhainews.com/n6427694.html
'utf-8' codec can't decode byte 0xc0 in position 1100: invalid start byte
753
754
---------------
754 产品销售 插播一条小广告：需要投资理财、各类保险、车险、贷款、买车买房买门面的朋友们！可以找我啦[奸笑][奸笑] ​
http://weibo.com/1642004624/FDpW3AF1v
substring not found
755
756
---------------
756 行业 河南保险业推进保险监管和服务实体经济全面升级
http://www.hninsure.com/xinxifabu/yejiedongtai/20847.html
substring not found
757
---------------
757 公司内部管理 蚂蚁招财宝都违约了这怎么样世界上还有靠谱的投资吗？
http://www.fjxwcbj.gov.cn/lanmuzt/9643.html
substring not found
758
759
760
---------------
760 行业 
		中午坐着无聊，算了一笔账：年薪3W，其中一家人保费8K，月定投500，完成各种任务费用约5K，合计1.9W，剩余1.1W，我和儿子平均每个月900块的生活费，他还平均一周一罐奶粉我……是怎么活下来的 ​
		
http://weibo.com/5932601877/FDq0RrSVU
substring not found
761
---------------
761 行业 【年底频现举牌，幕后推手在抄底？】一般而言，每年年底A股市场都会出现一波举牌潮，而与以往不同的是，2017年年底险资、私募基金举牌热情有所降温，而产业资本则持续升温，举牌多家上市公司。对于年底出现的举牌现象，私募观点是怎样的？私募排排网就此采访了几家私募公司，他们表达了各自不同的观点 ​
http://weibo.com/1667856794/FDq0Zsuiy
substring not f

In [6]:
raw_data.to_excel('data/raw_data/%s'%filename, index = False)

In [5]:
raw_data.head()

,label,title,content,url,title_url
0,行业,我国政策性信用保险承保金额突破5000亿美元-财急送网,全文 \n\r关键字 \n\r标题\n\r(视频)\n\r(视频)\n\r&lt;obje...,http://www.caijisong.com.cn/article/news-27344...,我国政策性信用保险承保金额突破5000亿美元
1,行业,因理论不足及司法配合不利 车险反欺诈任重道远,"产生错误的可能原因：\n""}}",http://www.gzjrw.com.cn/Item/335756.aspx,产生错误的可能原因：
2,行业,30余家互联网保险融资逾10亿 涉互联网保险但无牌照,"产生错误的可能原因：\n""}}",http://www.gzjrw.com.cn/Item/335761.aspx,产生错误的可能原因：
3,产品销售,短期趸交保险产品销售紧俏 一小时揽收超10亿元保费,"产生错误的可能原因：\n""}}",http://www.gzjrw.com.cn/Item/335762.aspx,产生错误的可能原因：
4,行业,税延型养老险“爽约”落地还须细化规则,"产生错误的可能原因：\n""}}",http://www.gzjrw.com.cn/Item/335760.aspx,产生错误的可能原因：


## label + title + content

In [6]:
filename = 'circ_class_combined_20180725_revise.xlsx'
raw_data = pd.read_excel('data/raw_data/%s'%filename)
print(raw_data.shape)
raw_data.head()

(2079, 5)


,label,title,content,url,title_url
0,噪音,做好规划不当月光族不做守财奴,一一妈妈说理财·出品\n (图片)\n\r月光族困惑：···这个月才过半，各种经费不够啊！...,https://xueqiu.com/1983578372/98551727,妈妈说理财: 做好规划不当月光族不做守财奴 一一妈妈说理财·出品 月光族困惑：···这个月才...
1,噪音,做到以下三点，汽车保费想不便宜都难,"参数错误\n\r微信公众平台运营中心\n""}}",http://mp.weixin.qq.com/s?__biz=MzA4ODYwMTczNg...,参数错误
2,噪音,做车险代理的前景如何 谊信合让你的财富事业迈上新高度,小本创业网\n\r»\n\r加盟项目资讯›\n\r»\n\r资讯中心\n\r»\n\r做车...,http://jxhb.91cy.cn/119134.shtml,做车险代理的前景如何 谊信合让你的财富事业迈上新高度
3,噪音,做保险想要高业绩，那就每天晚上8:30，来抢投保意向客户啊！,"参数错误\n\r微信公众平台运营中心\n""}}",http://mp.weixin.qq.com/s?__biz=MzI2NjUyMTUxNQ...,参数错误
4,噪音,坐顺风车出事了，车险理赔吗,"相关文章\n""}}",http://baijiahao.baidu.com/s?id=15887496924060...,坐顺风车出事了，车险理赔吗


# 对爬取数据进行清洗

In [162]:
raw_data_1=raw_data[~raw_data['title_url'].isin(['哎呀迷路了...','参数错误','抱歉！您访问的页面不存在！','内容被删除','404',
                                               '产生错误的可能原因：','你要找的新闻不存在，或者已经被管理员删除！','抱歉，您要访问的信息不存在或被删除',
                                               'error','nginx/1.10.3 (Ubuntu)','本帖已删除,或访问受限','腾讯新闻',
                                                '抱歉，指定的主题不存在或已被删除或正在被审核','此页面上的内容需要较新版本的 Adobe Flash Player。',
                                                'No such file or directory.','页面自动','文章出走，暂时未找到','很抱歉，没有找到相关信息！',
                                                '404：页面没有找到','很抱歉，此账号出现异常，暂时无法访问。','404 Not Found'])]
print(raw_data_1.shape)
raw_data_2=raw_data_1.dropna()#删除包含空值的数据
print(raw_data_2.dropna().shape)

(1765, 5)
(1659, 5)


In [163]:
raw_data_3=raw_data_2[~raw_data_2['url'].isin(
    raw_data_2[raw_data_2['content'].str.contains('"}}',na=False)&raw_data_2['url'].str.contains('toutiao.com',na=False)]['url'].tolist()
)]#去除 \n"}}0 和 (图片)"}}的数据
print(raw_data_3.shape)

(1473, 5)


# 保存清洗后的数据

In [161]:
raw_data_3.to_excel('data/raw_data/20180723_revise_clearn.xlsx', index = False)

In [ ]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_1_pre.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)